In [5]:
# Setup the Jupyter version of Dash
from dash import jupyter_dash, Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import custom Python middleware to handle database interactions
from CRUD import AnimalShelter

# Automatic date parsing generates many warnings which we can ignore
# Filter to avoid hiding critical warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize the JupyterDash application
jupyter_dash.default_mode="external"
app = Dash(__name__)

# Fixing column headers creates a problem where columns size to data but not to headers, which
# makes some column headers unreadable and sloppy.  This function generates a custom style to manage
# column widths based on the length of data in each column.
def create_conditional_style(df):
    # Define constants for padding and approximate pixel width per character
    PADDING = 24
    PIXEL_FOR_CHAR = 6
    
    # Initialize an empty list to store the style dictionary for each column
    style = []
    
    # Iterate over each column in the DataFrame
    for col in df.columns:
        
        # If the column data type is 'object', attempt to convert it to datetime
        # (useful for date columns stored as strings)
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])  # Convert to datetime if possible
            except ValueError:
                pass  # Ignore columns that can't be converted
        
        # Get all values in the column as a list
        col_list = df[col].values.tolist()
        
        # Ensure each item in the list is a string for consistent length measurement
        col_list = [s if isinstance(s, str) else str(s) for s in col_list]
        
        # Add the column name to the list to consider its length in width calculations
        col_list.append(col)
        
        # Calculate the length of the longest item in the column
        name_length = len(max(col_list, key=len))
        
        # Determine pixel width based on the longest string length, padding, and character width
        pixel = PADDING + round(name_length * PIXEL_FOR_CHAR)
        
        # Format pixel width as a string with 'px' suffix for CSS styling
        pixel = str(pixel) + 'px'
        
        # Check if the column contains string or boolean data and is not a datetime type
        # If true, align text to the left; otherwise, default alignment
        if (pd.api.types.infer_dtype(df[col]) == 'string' or pd.api.types.infer_dtype(df[col]) == 'boolean') and not pd.api.types.is_datetime64_any_dtype(df[col]):
            style.append({'if': {'column_id': col}, 'minWidth': pixel, 'textAlign': 'left'})
        else:
            style.append({'if': {'column_id': col}, 'minWidth': pixel})
    
    # Return the list of style dictionaries for the DataTable
    return style

# Establish database credentials
username = "aacuser"
password = "barkwoof"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Data preparation
df = pd.DataFrame.from_records(db.read_documents({}))
df.drop(columns=['_id'],inplace=True)

# Image encoding
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dashboard layout
app.layout = html.Div([
    # Header layout
    html.Center(html.B(html.H1('Joe Clancy - 11/21/2024 - CS499 - Enhanced Database Artifact'))),
    html.Hr(),
    html.Center(html.A(href="http://www.snhu.edu",children=[
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height='200', width='200')])),
    html.Div(),
    html.Hr(),

    # Data filters
    html.P('Filter for candidate animals:'),
    dcc.Dropdown(id='filter-type',
                 options=[{'label':'Water Rescue', 'value': 'water'},
                          {'label':'Mountain or Wilderness Rescue', 'value': 'mountain'},
                          {'label':'Disaster or Individual Tracking', 'value': 'disaster'},
                          {'label':'Custom Filter', 'value': 'custom'},
                          {'label':'No Filter', 'value': 'reset'}],
                 value='reset'),
    html.Hr(),

    # Filter options for custom filter
    # Hidden by default, becomes visible when custom filter is selected
    html.Div(id='custom-filter-options', style={'display': 'none'}, children=[
        html.P("Specify custom filter criteria:"),
        html.Label("Animal Type:"),
        dcc.Input(id='custom-animal-type', type='text', placeholder='e.g., Dog'),
        
        html.Label("Breed:"),
        dcc.Input(id='custom-breed', type='text', placeholder='e.g., Labrador'),

        html.Label("Sex:"),
        dcc.Input(id='custom-sex', type='text', placeholder='e.g., Intact Male'),
        
        html.Label("Minimum Age (weeks):"),
        dcc.Input(id='custom-age-min', type='number', placeholder='e.g., 20'),
        
        html.Label("Maximum Age (weeks):"),
        dcc.Input(id='custom-age-max', type='number', placeholder='e.g., 300'),
    ]),

    html.Br(),

    # Table layout
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single', # Allow one row to be selectable
                         style_cell_conditional=create_conditional_style(df), # Apply custom column widths
                         sort_action='native', # Enable sorting by column
                         sort_mode='single', # Only sort by one column at a time
                         fixed_rows={'headers':True}, # Fix column headers
                         selected_rows = [0], # Start with first row selected
                         page_size = 10, # Set maximum number of rows
                        ),

    html.Br(),
    
    # Message to display if no results are found
    html.Div(id='no-results-message', style={'color': 'red', 'display': 'none'}, children="No results found for the specified filter."),
    
    html.Br(),
    html.Hr(),
    
    # Configure graph and map to be side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
            ])
])

# Callbacks

# Show/hide custom filter options based on filter selection
@app.callback(Output('custom-filter-options', 'style'),
              Input('filter-type', 'value'))

def toggle_custom_filter(filter_type):
    if filter_type == 'custom':
        return {'display': 'block'}
    return {'display': 'none'}

# Filter data based on drop-down selection
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','selected_rows'),
              Output('no-results-message', 'style')],
                  [Input('filter-type', 'value'),
                   Input('custom-animal-type', 'value'),
                   Input('custom-breed', 'value'),
                   Input('custom-sex', 'value'),
                   Input('custom-age-min', 'value'),
                   Input('custom-age-max', 'value')])

def update_dashboard(filter_type, custom_animal_type, custom_breed, custom_sex, custom_age_min, custom_age_max):
    # If the filter type has not been set, do not run
    if not filter_type:
        return

    # Default values for custom filter, in case inputs have not been initialized
    if custom_animal_type == None: custom_anmial_type = ""
    if custom_breed == None: custom_breed = ""
    if custom_sex == None: custom_sex = ""
    if custom_age_min == None: custom_age_min = 0
    if custom_age_max == None: custom_age_max = 99999
    
    # Filter for animals that meet 'Water Rescue' criteria
    if filter_type == 'water':
        query = {"animal_type":"Dog",
         "$or":[
             {"breed":{"$regex": "Labrador"}},
             {"breed":{"$regex": "Chesa Bay"}},
             {"breed":{"$regex": "Newfoundland"}}], 
         "sex_upon_outcome":"Intact Female",
         "age_upon_outcome_in_weeks":{"$gte":26},
         "age_upon_outcome_in_weeks":{"$lte":156}}

    # Filter for animals that meet 'Mountain Rescue' criteria
    elif filter_type == 'mountain':       
        query = {"animal_type":"Dog",
         "$or":[
             {"breed":{"$regex": "German Shepherd"}},
             {"breed":{"$regex": "Alaskan Malamute"}},
             {"breed":{"$regex": "Siberian Husky"}},
             {"breed":{"$regex": "Rottweiler"}},
             {"breed":{"$regex": "Old English Sheepdog"}}], 
         "sex_upon_outcome":"Intact Male",
         "age_upon_outcome_in_weeks":{"$gte":26},
         "age_upon_outcome_in_weeks":{"$lte":156}}

    # Filter for animals that meet 'Disaster Rescue' criteria
    elif filter_type == 'disaster':        
        query = {"animal_type":"Dog",
         "$or":[
             {"breed":{"$regex": "German Shepherd"}},
             {"breed":{"$regex": "Doberman Pinscher"}},
             {"breed":{"$regex": "Golden Retriever"}},
             {"breed":{"$regex": "Rottweiler"}},
             {"breed":{"$regex": "Bloodhound"}}], 
         "sex_upon_outcome":"Intact Male",
         "age_upon_outcome_in_weeks":{"$gte":20},
         "age_upon_outcome_in_weeks":{"$lte":300}}

    # Filter for animals meeting custom requirements
    # options "i" for case insensitive
    elif filter_type == 'custom':        
        query = {"animal_type": {"$regex": custom_animal_type, "$options": "i"},
         "breed":{"$regex": custom_breed, "$options": "i"}, 
         "sex_upon_outcome": {"$regex": custom_sex, "$options": "i"},
         "age_upon_outcome_in_weeks":{"$gte":custom_age_min},
         "age_upon_outcome_in_weeks":{"$lte":custom_age_max}}

    else:
        query = {}

    # Query records from database
    df = pd.DataFrame.from_records(db.read_documents(query))

    # If no records are found
    if df.empty:
        # Show "No results found" message and return empty data
        return [], [], {'color': 'red', 'display': 'block'}
    
    df.drop(columns=['_id'],inplace=True)

    # Format records as dict
    data=df.to_dict('records')
    
    return data, [0], {'color': 'red', 'display': 'none'}

# Populate graph based on data visible in table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    if not viewData:
        return

    dff = pd.DataFrame.from_dict(viewData)

    # Truncate data to only MAX_BREEDS breeds, group the rest as "Other"
    MAX_BREEDS = 10
    
    if len(dff) > MAX_BREEDS:
        frequency = dff['breed'].value_counts()
        
        top_values = frequency.nlargest(MAX_BREEDS).index  # Get the top `n` values
        dff['breed'] = dff['breed'].apply(lambda x: x if x in top_values else 'Other')
    
    return [
        dcc.Graph(
            id = 'pie_chart',            
            figure = px.pie(dff, names='breed')
        )    
    ]
    
# Highlight a table cell when user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    # If column has not been selected, do not run
    if not selected_columns:
        return

    # Define highlight color
    BG_COLOR = '#D2F3FF' # Red
    
    return [{
        'if': { 'column_id': i },
        'background_color': BG_COLOR
    } for i in selected_columns]

# Center chart on geo-coordinates associated with selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    # If parameters are not configured, do not run
    if viewData is None:
        return
    elif index is None:
        return
    elif len(index) == 0:
        return

    # Define default map parameter
    ZOOM_LEVEL = 10

    # Convert viewData to dataframe
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, convert list to a row index
    row = index[0]

    # Get animal information from dataframe
    # DO NOT USE absolute column indices, they are inconsistent.
    animalLat = dff.iloc[row, dff.columns.get_loc('location_lat')]
    animalLon = dff.iloc[row, dff.columns.get_loc('location_long')]
    animalBreed = dff.iloc[row, dff.columns.get_loc('breed')]
    animalName = dff.iloc[row, dff.columns.get_loc('name')]
        
    # Austin TX is at [30.75,-97.48]
    return [
        # Center map on animal coordinates
        dl.Map(style={'width': '750px', 'height': '500px'}, center=[animalLat,animalLon], zoom=ZOOM_LEVEL, children=[
            dl.TileLayer(id="base-layer-id"),
            # Place marker with animal details
            dl.Marker(position=[animalLat,animalLon], children=[
                dl.Tooltip(animalBreed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animalName)
                ])
            ])
        ])
    ]

# Run server
app.run_server(debug=True)


C:\Users\clanc\AppData\Local\Temp\ipykernel_16020\4186767534.py:26: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\clanc\AppData\Local\Temp\ipykernel_16020\4186767534.py:26: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\clanc\AppData\Local\Temp\ipykernel_16020\4186767534.py:26: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\clanc\AppData\Local\Temp\ipykernel_16020\4186767534.py:26: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please spe

Dash app running on http://127.0.0.1:8050/
